In [2]:
import torch 
from torch import nn 
from torch.nn import functional as F

In [3]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
X = torch.rand(2, 20)
net(X).shape

/home/shkim/anaconda3/envs/d2l/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 10])

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.out = nn.LazyLinear(10)
        
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [5]:
net = MLP()
net(X).shape

torch.Size([2, 10])

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__() 
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)

    def forward(self, X):
        for module in self.children():
            X = module(X) 
        return X

In [7]:
net = MySequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10)) 
net(X).shape

torch.Size([2, 10])

In [16]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.randn((20,20))
        self.linear = nn.LazyLinear(20)
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(X@self.rand_weight+1)
        X = self.linear(X)
        while X.abs().sum()>1:
            X/=2
        return X.sum()

In [19]:
net = FixedHiddenMLP()
net(X)

tensor(-0.3005, grad_fn=<SumBackward0>)

In [23]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(64), nn.ReLU(),
                                  nn.LazyLinear(32), nn.ReLU())
        self.linear = nn.LazyLinear(16)
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.LazyLinear(20), FixedHiddenMLP())
chimera(X)

tensor(0.3502, grad_fn=<SumBackward0>)

In [27]:
class Parallel(nn.Module):
    def __init__(self, net1, net2):
        super.__init__()
        self.net1 = net1
        self.net2 = net2
    def forward(self, x):
        output1 = self.net1(x)
        output2 = self.net2(x)
        return torch.cat(output1, output2, dim=1)

In [32]:
class Block(nn.Module):
    def __init__(self, input, output):
        super().__init__()
        self.layer = nn.Linear(input, output)
        
    def forward(self, X):
        return self.layer(X)
    
def Factory(input, output, number):
    modules = nn.Sequential()
    for i in range (number):
         modules.add_module(str(i), Block(input, output))
    return modules

net = Factory(10, 1, 2)
net
     

Sequential(
  (0): Block(
    (layer): Linear(in_features=10, out_features=1, bias=True)
  )
  (1): Block(
    (layer): Linear(in_features=10, out_features=1, bias=True)
  )
)

In [39]:
import torch
from torch import nn

net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))
X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

In [42]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[-0.1937,  0.0997, -0.1211, -0.1833,  0.1379, -0.2170,  0.0321, -0.2171]])),
             ('bias', tensor([0.3291]))])

In [44]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([0.3291]))

In [46]:
net[2].weight.grad == None

True

In [48]:
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

In [52]:
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0,0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [70]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(64), nn.ReLU(),
                                  nn.LazyLinear(32), nn.ReLU())
        self.linear = nn.LazyLinear(16)
    def forward(self, X):
        return self.linear(self.net(X))
    
nest = NestMLP()
nest(X)
[(name, param.size) for name, param in nest.named_parameters()]

/home/shkim/anaconda3/envs/d2l/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


[('net.0.weight', <function Parameter.size>),
 ('net.0.bias', <function Parameter.size>),
 ('net.2.weight', <function Parameter.size>),
 ('net.2.bias', <function Parameter.size>),
 ('linear.weight', <function Parameter.size>),
 ('linear.bias', <function Parameter.size>)]

In [71]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.out = nn.LazyLinear(10)
        
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))
    
class SharedMLP(nn.Module):
    def __init__(self):
        super.__init__()
        shared = nn.Linear(256, 256)
        self.hidden1 = nn.Linear(256, 128)
        self.hidden2 = nn.Linear(256, 128)
        self.out = nn.Linear(128, 10)
    def forward(self, X):
        shared_out = self.shared(X)
        h1_out = F.relu(self.hidden1(shared_out))
        h2_out = F.relu(self.hidden2(shared_out))
        tot_out = h1_out+h2_out
        return self.out(tot_out)

In [75]:
import torch
from torch import nn

net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(), nn.LazyLinear(1))
X = torch.rand(size=(2,4))
net(X).shape

torch.Size([2, 1])

In [77]:
def init_normal(module):
    if type(module) == nn.Linear:
        nn.init.normal_(module.weight, mean =0, std=0.01)
        nn.init.zeros_(module.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0078, -0.0023, -0.0062,  0.0056]), tensor(0.))

In [79]:
def init_constant(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight,1)
        nn.init.zeros_(module.bias)
        
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [81]:
def init_xavier(module):
    if type(module) == nn.Linear:
        nn.init.xavier_uniform_(module.weight)
        
def init_42(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 42)    

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data[0])

tensor([ 0.6163, -0.3481, -0.1217, -0.2322])
tensor([42., 42., 42., 42., 42., 42., 42., 42.])


In [85]:
def my_init(module):
    if type(module) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in module.named_parameters()][0])
        nn.init.uniform_(module.weight, -10, 10)
        module.weight.data *= module.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  0.0000,  0.0000, -9.5718],
        [-0.0000,  0.0000, -9.4475,  9.0504]], grad_fn=<SliceBackward0>)

In [86]:
import torch 
from torch import nn 
from d2l import torch as d2l

net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
net[0].weight

/home/shkim/anaconda3/envs/d2l/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


<UninitializedParameter>

In [88]:
X = torch.rand(2, 20) 
net(X)
net[0].weight.shape


torch.Size([256, 20])

In [90]:
@d2l.add_to_class(d2l.Module) #@save 
def apply_init(self, inputs, init=None):
    self.forward(*inputs) 
    if init is not None:
        self.net.apply(init)

In [91]:
import torch 
from torch import nn 
from torch.nn import functional as F 
from d2l import torch as d2l

In [92]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,X):
        return X-X.mean()


In [94]:
layer = CenteredLayer()
layer(torch.tensor([1.0,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [97]:
net = nn.Sequential(nn.LazyLinear(128), CenteredLayer())
Y = net(torch.rand(4, 8)) 
Y.mean()

tensor(4.6566e-10, grad_fn=<MeanBackward0>)

In [98]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
        
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data)+self.bias.data
        return F.relu(linear)
        

In [100]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.5841, -0.6845, -0.9922],
        [ 0.1617, -0.1921, -0.8716],
        [ 0.5900,  0.5088,  1.3721],
        [-1.3681,  0.2260,  0.1873],
        [-1.3433, -0.2587,  0.3184]], requires_grad=True)

In [102]:
linear(torch.rand(2,5))

tensor([[0.2665, 1.4422, 0.4275],
        [0.0000, 1.4522, 0.8904]])

In [105]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8,1))
net(torch.rand(2,64))

tensor([[0.],
        [0.]])

In [106]:
class TensorLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(input_dim, input_dim, output_dim))
    def forward(self,X):
        X_expanded = X.unsqueeze(2)  
        X_expanded = X_expanded * X.unsqueeze(1)
        output = torch.einsum('bij,ijk->bk', X_expanded, self.weight)
        return output
input_dim = 5
output_dim = 3
batch_size = 2

layer = TensorLayer(input_dim, output_dim)
X = torch.randn(batch_size, input_dim)
output = layer(X)

print("Input:")
print(X)
print("\nOutput:")
print(output)

Input:
tensor([[-0.4680,  0.4408, -0.9298,  0.3213,  0.3195],
        [ 1.0134,  0.5675,  0.1367,  0.6797, -0.7000]])

Output:
tensor([[ 1.5608,  0.5770,  1.9925],
        [ 5.6716, -0.1172,  0.1010]], grad_fn=<ViewBackward0>)


In [110]:
import torch.fft

class FourierLayer(nn.Module):
    def __init__(self):
        super(FourierLayer, self).__init__()

    def forward(self, X):
        X_ft = torch.fft.fft(X, dim=-1)
        return X_ft[..., :X_ft.shape[-1] // 2]

In [113]:
import torch 
from torch import nn 
from torch.nn import functional as F

x = torch.arange(4) 
torch.save(x, 'x-file')

x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [115]:
y = torch.zeros(4) 
torch.save([x, y],'x-files') 
x2, y2 = torch.load('x-files') 
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [117]:
mydict = {'x': x, 'y': y} 
torch.save(mydict, 'mydict') 
mydict2 = torch.load('mydict') 
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [120]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.output = nn.LazyLinear(10)
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2,20))
Y = net(X)

torch.save(net.state_dict(), 'mlp.params')

# to recover model
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

/home/shkim/anaconda3/envs/d2l/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


MLP(
  (hidden): LazyLinear(in_features=0, out_features=256, bias=True)
  (output): LazyLinear(in_features=0, out_features=10, bias=True)
)

In [123]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [125]:
import torch 
from torch import nn 
from d2l import torch as d2l

In [126]:
def cpu():
    return torch.device('cpu')
def gpu(i=0):
    return torch.device(f'cuda:{i}')

cpu(), gpu(), gpu(1)

(device(type='cpu'),
 device(type='cuda', index=0),
 device(type='cuda', index=1))

In [128]:
def num_gpus():
    return torch.cuda.device_count()
num_gpus()

8

In [131]:
def try_gpu(i=0):
    if num_gpus() >= i+1:
        return gpu(i)
    return cpu()
def try_all_gpus():
    return [gpu(i) for i in range(num_gpus())]


try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0),
  device(type='cuda', index=1),
  device(type='cuda', index=2),
  device(type='cuda', index=3),
  device(type='cuda', index=4),
  device(type='cuda', index=5),
  device(type='cuda', index=6),
  device(type='cuda', index=7)])

In [132]:
x = torch.tensor([1,2,3])
x.device

device(type='cpu')

In [134]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [137]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.3966, 0.1175, 0.4929],
        [0.5484, 0.4473, 0.6310]], device='cuda:1')

In [141]:
Z = X.cuda(1)
print(X)
print(Z)

Y + Z

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:1')


tensor([[1.3966, 1.1175, 1.4929],
        [1.5484, 1.4473, 1.6310]], device='cuda:1')

In [143]:
Z.cuda(1) is Z

True

In [148]:
net = nn.Sequential(nn.LazyLinear(1))
net = net.to(device=try_gpu())

net(X)

tensor([[0.6816],
        [0.6816]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [149]:
net[0].weight.data.device

device(type='cuda', index=0)

In [158]:
def __init__(self, max_epochs, num_gpus=0, gradient_clip_val=0):
    self.save_hyperparameters()
    self.gpus = [d2l.gpu(i) for i in range(min(num_gpus, d2l.num_gpus()))]
    
def prepare_batch(self,batch):
    if self.gpus:
        batch=[a.to(self.gpus[0]) for a in batch]
    return batch
def prepare_model(self, model):
    model.trainer = self
    model.board.xlim = [0, self.max_epochs]
    if self.gpus:
        model.to(self.gpus[0])
    self.model = model